In [8]:
import pandas as pd
import urllib.request
import json
import sqlite3

In [2]:
#read data from today
df = pd.read_json("https://tsimobile.viarail.ca/data/allData.json")
#transpose data
df_t = df.transpose()
#shift train name into column
df_t = df_t.reset_index()

# Pull Daily Data from JSON

In [3]:
via_day_data = df_t[['times','index','departed','arrived', 'from', 'to', ]]

# Initialize an empty list to store DataFrames with IDs
flattened_dfs = []

In [4]:
# Iterate over each row in x
for idx, row in via_day_data.iterrows():

    # Iterate over each DataFrame in the sublist
    for df in row['times']:
        # Add the ID as a column to the DataFrame
        df['ID'] = row['index']
        df['departed'] = row['departed']
        df['arrived'] = row['arrived']
        df['from'] = row['from']
        df['to'] = row['to']
        # Append the DataFrame to the list
        flattened_dfs.append(df)

test_2 = pd.DataFrame(flattened_dfs)

#Clean up datetimes
test_2['estimated'] = pd.to_datetime(test_2['estimated'], errors = 'coerce')
test_2['scheduled'] = pd.to_datetime(test_2['scheduled'], errors = 'coerce')

In [10]:
via_day_data_clean = test_2[['station','code', 'ID','scheduled','estimated','departed','diffMin', 'arrived','from','to']]

In [31]:
con = sqlite3.connect("via_data.db")
cur = con.cursor()

In [33]:
sqlite_insert_with_param = """INSERT INTO tbl_location(LOCATION, LOCATION_CODE) VALUES (?, ?) ON CONFLICT(LOCATION) DO NOTHING;"""

# Iterate over each row in x
for idx, row in via_day_data_clean.iterrows():

    location_i = row['station']
    location_code_i = row['code']

    print(location_i)
    data_tuple = (location_i, location_code_i)

    cur.execute(sqlite_insert_with_param, data_tuple)

con.commit
cur.close()    
    

Toronto


OperationalError: database is locked

In [24]:
con.commit

<function Connection.commit()>

In [34]:
cur.execute("SELECT * FROM tbl_location")
rows = cur.fetchall()
for row in rows:
    print(row)

In [29]:
cur.close()   